# 국방조달

In [1]:
# don't need to load extension everytime. 
%load_ext autoreload 

# This needs to be executed everytime.  
%autoreload 2

In [2]:


from org.d2b.d2b import D2B
from account import account_get
d2b_id = account_get("d2b", "id")
d2b_pw = account_get("d2b", "pw")
d2b_user = account_get("d2b", "user")
d2b_cert = account_get("d2b", "cert")
d2b = D2B(d2b_id, d2b_pw, d2b_user, d2b_cert, headless=False)



In [3]:

# This needs to be executed everytime.  
%autoreload 2

d2b.login()

In [ ]:
# 2023UMM032323380-01 , price=  79,417,016
d2b.participate("2023UMM032323380-01", "79417016")